CBraMod 라는 사전학습 모델을 불러와 HBN-EEG 데이터로 downstream task supervised learning 수행

## 라이브러리 및 CBraMod 설치

In [1]:
!pip install mne torch pandas tqdm einops

In [2]:
!git clone https://github.com/wjq-learning/CBraMod.git
!pip install -r CBraMod/requirements.txt

fatal: destination path 'CBraMod' already exists and is not an empty directory.


In [ ]:
import os, sys, json, math, random, gc
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import mne
from multiprocessing import Pool, cpu_count

# ============ 경로/하이퍼파라미터 ============
BIDS_ROOT = "/data5/open_data/HBN/EEG_BIDS" 
CBRAMOD_PATH = "/home/RA/EEG_Challenge/Challenge2/CBraMod"  
PRETRAINED_MODEL_PATH = "/home/RA/hf_models/cbramod_pretrained.pth"

SAVE_ROOT = "./c2_lightft_artifacts"
CACHE_FILELIST = os.path.join(SAVE_ROOT, "cache_valid_eegs.json")
NPZ_DIR = os.path.join(SAVE_ROOT, "npz")
CKPT_DIR = os.path.join(SAVE_ROOT, "checkpoints")

os.makedirs(SAVE_ROOT, exist_ok=True)
os.makedirs(NPZ_DIR, exist_ok=True)
os.makedirs(CKPT_DIR, exist_ok=True)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); 
if DEVICE == "cuda": torch.cuda.manual_seed_all(SEED)

# ============ CBraMod import ============
if CBRAMOD_PATH not in sys.path:
    sys.path.append(CBRAMOD_PATH)
from models.cbramod import CBraMod  # repo의 모델 구현 사용

Using device: cuda


## 필요 함수 정의

BIDS에서 15분 이상 세션만 골라 200Hz/30s로 윈도우

In [ ]:
def load_all_participants(bids_root, datasets):
    cols = ['participant_id', 'p_factor', 'internalizing', 'externalizing', 'attention']
    allp = []
    print(f"\nLoading participant labels from datasets: {datasets}")
    for ds in tqdm(datasets, desc="Reading participants.tsv"):
        pfile = os.path.join(bids_root, ds, "participants.tsv")
        if not os.path.exists(pfile): 
            continue
        try:
            df = pd.read_csv(pfile, sep="\t")
            if all(c in df.columns for c in cols):
                allp.append(df[cols].copy())
        except Exception as e:
            print(f"  ⚠️ {pfile}: {e}")
            continue
    if not allp:
        print("⚠️ No valid participants.tsv found.")
        return pd.DataFrame(columns=cols).set_index("participant_id")
    df = pd.concat(allp, ignore_index=True)
    df["participant_id"] = df["participant_id"].str.strip()     # 'sub-000001' 형태 유지
    df = df.dropna(subset=cols).drop_duplicates("participant_id")
    print(f"  ✅ Loaded participants: {len(df)}")
    return df.set_index("participant_id")

def collect_raw_files_cached(bids_root, datasets, min_duration_sec=900):
    """15분(900s) 이상 raw .set만 수집하고 JSON 캐시."""
    if os.path.exists(CACHE_FILELIST):
        print(f"📦 Using cached EEG list: {CACHE_FILELIST}")
        with open(CACHE_FILELIST, "r") as f:
            return json.load(f)

    print(f"\n🔍 Scanning BIDS_ROOT for >= {min_duration_sec/60:.1f} min EEGs ...")
    all_files = []
    for ds in datasets:
        all_files.extend(glob(os.path.join(bids_root, ds, "sub-*", "eeg", "sub-*_eeg.set")))
    valid = []
    for eeg_path in tqdm(all_files, desc="Checking durations"):
        try:
            raw = mne.io.read_raw_eeglab(eeg_path, preload=False, verbose="ERROR")
            duration = raw.n_times / raw.info["sfreq"]
            if duration >= min_duration_sec:
                valid.append(eeg_path)
        except Exception:
            continue
    with open(CACHE_FILELIST, "w") as f:
        json.dump(valid, f)
    print(f"  ✅ Cached {len(valid)} EEGs to {CACHE_FILELIST}")
    return valid

# ============ 전처리 -> 윈도우 -> .npz (병렬) ============
def subject_id_from_path(p):
    # sub-000001_... 형태
    return os.path.basename(p).split("_")[0]  # 'sub-000001'

def preprocess_single_eeg(eeg_path):
    """raw .set -> 200Hz, 0.3-75Hz, 30s 윈도우 (N, C, 6000) 를 sub-XXXX.npz로 저장"""
    try:
        sid = subject_id_from_path(eeg_path)
        out = os.path.join(NPZ_DIR, f"{sid}.npz")
        if os.path.exists(out):
            return out

        raw = mne.io.read_raw_eeglab(eeg_path, preload=True, verbose="ERROR")
        raw.filter(0.3, 75., n_jobs=1, verbose="ERROR")
        raw.notch_filter([60], n_jobs=1, verbose="ERROR")
        raw.resample(200, npad="auto", verbose="ERROR")

        X = raw.get_data().astype(np.float32)      # (C, T)
        C, T = X.shape
        win = 30 * 200                              # 6000
        n = T // win
        if n == 0: 
            return None
        X = X[:, :n*win].reshape(C, n, win).transpose(1,0,2)  # (N, C, 6000)
        np.savez_compressed(out, X=X)
        return out
    except Exception as e:
        return None

def preprocess_all_eegs(eeg_files, n_jobs=None):
    n_jobs = n_jobs or max(1, cpu_count()-2)
    print(f"\n⚙️ Preprocessing {len(eeg_files)} EEGs with {n_jobs} workers -> {NPZ_DIR}")
    with Pool(n_jobs) as p:
        results = list(tqdm(p.imap(preprocess_single_eeg, eeg_files), total=len(eeg_files)))
    results = [r for r in results if r is not None]
    print(f"  ✅ Saved {len(results)} subject .npz files")
    return results

externalizing만 예측하는 것으로 바뀐 게 맞는지 확인

## 데이터셋 정의

In [ ]:
class HBNWindowDataset(Dataset):
    def __init__(self, npz_files, label_df, zscore=True, pick_one_window=True):
        self.paths = npz_files
        self.label_df = label_df  # index: 'sub-XXXXXX'
        self.zscore = zscore
        self.pick_one = pick_one_window

    def __len__(self): 
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        sid = os.path.basename(path).split(".")[0]  # 'sub-XXXXXX'
        obj = np.load(path)
        X = obj["X"]  # (N, C, 6000)
        if self.pick_one:
            i = np.random.randint(0, X.shape[0])
            x = X[i]  # (C, 6000)
        else:
            # 모든 윈도우 평균(robust)로 subject 대표값 만들기
            x = X.mean(axis=0)

        if self.zscore:
            # 채널별/시간 전체 Z-score (subject 단위 고정 스케일)
            mu = x.mean(axis=1, keepdims=True)
            sd = x.std(axis=1, keepdims=True) + 1e-6
            x = (x - mu) / sd

        y = self.label_df.loc[sid, ["p_factor","internalizing","externalizing","attention"]].values.astype(np.float32)
        return {"eeg": torch.tensor(x, dtype=torch.float32), 
                "label": torch.tensor(y, dtype=torch.float32)}

## 모델 정의 (head 부분만)

In [ ]:
class EEGRegressor(nn.Module):
    """CBraMod 인코더 + 전역 평균 + 얕은 회귀 헤드(4D)"""
    def __init__(self, encoder: nn.Module, n_outputs=4, dropout=0.1):
        super().__init__()
        self.encoder = encoder
        # encoder.proj_out은 feature 차원 정렬에 영향 → 라이트 FT에서는 Identity로 사용 (features만)
        self.encoder.proj_out = nn.Identity()

        # 더미 입력으로 feature dim 확인
        with torch.no_grad():
            dummy = torch.randn(1, 128, 6000)  # (B, C, T) , 채널 128 가정
            feats = self.encoder( dummy.reshape(1, 128, 30, 200) )  # (B, C, N, D) 형태 가정
            # 위 CBraMod 포맷에 따라 변환: (B, C, S, P) 입력일 때 모델 내부 패칭을 생략하려면
            # 리쉐이프 없이 encoder가 패칭을 수행하는 구현이라면 아래를 encoder(dummy)로 변경.
            if feats.ndim == 4:
                D = feats.shape[-1]
            elif feats.ndim == 3:
                D = feats.shape[-1]
            else:
                raise RuntimeError("Unknown CBraMod output shape.")
        self.head = nn.Sequential(
            nn.LayerNorm(D),
            nn.Linear(D, D*2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(D*2, n_outputs)
        )

    def forward(self, x):
        # x: (B, C, 6000) -> (B, C, S=30, P=200) 로 패칭 후 encoder에 투입
        b, c, t = x.shape
        assert t % 200 == 0, "Input T must be multiple of 200 (30s @200Hz -> 6000)"
        s = t // 200
        x = x.view(b, c, s, 200)
        feats = self.encoder(x)            # (B, C, S, D) or (B, S, D)
        if feats.ndim == 4:
            feats = feats.mean(dim=(1,2))  # (B, D)
        else:
            feats = feats.mean(dim=1)      # (B, D)
        return self.head(feats)            # (B, 4)

## 학습 코드

In [ ]:
def train_loop(train_loader, val_loader, model, epochs=15, lr=1e-4, wd=5e-4, save_dir=CKPT_DIR):
    model = model.to(DEVICE)
    scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))
    optim = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, weight_decay=wd)
    sched = torch.optim.lr_scheduler.CosineAnnealingLR(optim, T_max=epochs, eta_min=1e-6)
    crit = nn.MSELoss()

    best = float("inf")
    for ep in range(1, epochs+1):
        model.train()
        tr_loss = 0.0
        for batch in tqdm(train_loader, desc=f"Train [{ep}/{epochs}]"):
            x = batch["eeg"].to(DEVICE)     # (B, C, 6000)
            y = batch["label"].to(DEVICE)   # (B, 4)
            optim.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):
                yhat = model(x)
                loss = crit(yhat, y)
            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optim); scaler.update()
            tr_loss += loss.item() * len(x)

        tr_loss /= len(train_loader.dataset)

        model.eval()
        va_loss = 0.0
        with torch.no_grad(), torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):
            for batch in val_loader:
                x = batch["eeg"].to(DEVICE)
                y = batch["label"].to(DEVICE)
                yhat = model(x)
                loss = crit(yhat, y)
                va_loss += loss.item() * len(x)
        va_loss /= len(val_loader.dataset)
        sched.step()

        print(f"Epoch {ep:02d} | train MSE {tr_loss:.4f} | val MSE {va_loss:.4f} | lr {sched.get_last_lr()[0]:.2e}")

        if va_loss < best:
            best = va_loss
            path = os.path.join(save_dir, "c2_lightft_best.pt")
            torch.save({"model_state_dict": model.state_dict(),
                        "val_mse": best, "epoch": ep}, path)
            print(f"  💾 saved: {path}")

    last = os.path.join(save_dir, "c2_lightft_last.pt")
    torch.save({"model_state_dict": model.state_dict(), "epoch": epochs}, last)
    print(f"  💾 saved: {last}")

*현재는 회귀 헤드만 학습 -> 사전 학습 모델의 모든 레이어를 freeze

In [ ]:
def main():
    print(f"Using device: {DEVICE}")
    # 스플릿 (예: Release5 검증)
    all_sets = [f"ds00{i}" for i in range(5505, 5517)]
    train_sets = [s for s in all_sets if s != "ds005509"]
    val_sets   = ["ds005509"]

    # 라벨
    df_train = load_all_participants(BIDS_ROOT, train_sets)
    df_val   = load_all_participants(BIDS_ROOT, val_sets)

    # 파일 수집(캐시)
    valid_train = collect_raw_files_cached(BIDS_ROOT, train_sets, min_duration_sec=900)
    valid_val   = collect_raw_files_cached(BIDS_ROOT, val_sets,   min_duration_sec=900)

    # 전처리 → npz (병렬, 1회만)
    train_npz = preprocess_all_eegs(valid_train)
    val_npz   = preprocess_all_eegs(valid_val)

    # 라벨과 교집합만 사용
    train_npz = [p for p in train_npz if os.path.basename(p).split(".")[0] in df_train.index]
    val_npz   = [p for p in val_npz   if os.path.basename(p).split(".")[0] in df_val.index]
    print(f"Train NPZ: {len(train_npz)}, Val NPZ: {len(val_npz)}")

    # DataLoader
    num_workers = min(8, max(2, cpu_count()//2))
    train_loader = DataLoader(
        HBNWindowDataset(train_npz, df_train, zscore=True, pick_one_window=True),
        batch_size=16, shuffle=True, num_workers=num_workers, pin_memory=True
    )
    val_loader = DataLoader(
        HBNWindowDataset(val_npz, df_val, zscore=True, pick_one_window=True),
        batch_size=16, shuffle=False, num_workers=num_workers, pin_memory=True
    )

    # 모델 구성: CBraMod + 라이트 FT
    encoder = CBraMod().to(DEVICE)
    state = torch.load(PRETRAINED_MODEL_PATH, map_location=DEVICE)
    encoder.load_state_dict(state, strict=False)

    # 백본 거의 고정(라이트 FT) — 필요 시 일부 블록만 풀기
    for p in encoder.parameters():
        p.requires_grad = False
    # 예: encoder의 마지막 레이어만 학습하고 싶다면 필요한 모듈만 requires_grad=True로 설정

    model = EEGRegressor(encoder, n_outputs=4, dropout=0.1)
    for p in model.head.parameters():  # 회귀 헤드만 학습
        p.requires_grad = True

    train_loop(train_loader, val_loader, model, epochs=15, lr=1e-4, wd=5e-4, save_dir=CKPT_DIR)


Loading participant labels from: ['ds005505', 'ds005506', 'ds005507', 'ds005508', 'ds005510', 'ds005511', 'ds005512', 'ds005513', 'ds005514', 'ds005515', 'ds005516']


Loading participants.tsv: 100%|██████████| 11/11 [00:00<00:00, 29.28it/s]


Loading participant labels from: ['ds005509']


Loading participants.tsv: 100%|██████████| 1/1 [00:00<00:00, 25.39it/s]


Assigning files to train/val sets: 100%|██████████| 25944/25944 [00:00<00:00, 371219.38it/s]



Creating Datasets and DataLoaders...


Indexing preprocessed files:   0%|          | 8/22425 [00:01<38:33,  9.69it/s]  /home/RA/.local/lib/python3.10/site-packages/pymatreader/utils.py:296: UserWarning: pymatreader cannot import Matlab string variables. Please convert these variables to char arrays in Matlab.
  warn(
Indexing preprocessed files: 100%|██████████| 3158/3158 [06:49<00:00,  7.72it/s]



Loading pre-trained model...
Pre-trained weights loaded successfully!


In [ ]:
if __name__ == "__main__":  
    main()